In [213]:
import copy

In [214]:
def reallocate(banks):
    banks_copy = copy.copy(banks)
    
    blocks, index = max(banks), banks.index(max(banks))
    banks_copy[index] = 0
    
    while blocks:
        banks_copy[(index + 1) % len(banks)] += 1
        index += 1
        blocks -= 1
        
    return banks_copy

In [215]:
def cycle(banks):
    b = copy.copy(banks)
    seen_states = [b]
    
    while True:
        b = reallocate(b)
        
        if b in seen_states:
            return len(seen_states), len(seen_states) - seen_states.index(b)
            
        seen_states.append(b)

In [216]:
cycle([0, 2, 7, 0])

(5, 4)

In [217]:
cycle([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

(12841, 8038)

In [225]:
%prun cycle([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

In [219]:
def cycle_with_strings(banks):
    b = copy.copy(banks)
    b_string = ','.join([str(x) for x in b])
    seen_states = [b_string]
    
    while True:
        b = reallocate(b)
        b_string = ','.join([str(x) for x in b])
        
        if b_string in seen_states:
            return len(seen_states), len(seen_states) - seen_states.index(b_string)
            
        seen_states.append(b_string)

In [220]:
%prun cycle_with_strings([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

In [221]:
def cycle_with_strings_and_sets(banks):
    b = copy.copy(banks)
    b_string = ','.join([str(x) for x in b])
    seen_states = set([b_string])
    state_position = {}
    
    while True:
        b = reallocate(b)
        b_string = ','.join([str(x) for x in b])
        
        if b_string in seen_states:
            return len(seen_states), len(seen_states) - state_position[b_string]

        state_position[b_string] = len(seen_states)
        seen_states.add(b_string)

In [222]:
%prun cycle_with_strings_and_sets([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5])

In [224]:
print(cycle([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5]))
print(cycle_with_strings([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5]))
print(cycle_with_strings_and_sets([4,10,4,1,8,4,9,14,5,1,14,15,0,15,3,5]))

(12841, 8038)
(12841, 8038)
(12841, 8038)
